## Analyzing Weather Data from Open-Meteo API

| Column                                 | Description                                                                                                   |
|----------------------------------------|---------------------------------------------------------------------------------------------------------------|
| time                                   | The time of the weather observation.                                                                          |
| weathercode                            | The weather condition represented by a numeric code.                                                           |
| temperature_2m_max                     | The maximum air temperature at 2 meters above ground.                                                         |
| temperature_2m_min                     | The minimum air temperature at 2 meters above ground.                                                         |
| temperature_2m_mean                    | The mean air temperature at 2 meters above ground.                                                            |
| apparent_temperature_max               | The maximum apparent temperature, which is the perceived feels-like temperature combining wind chill factor, relative humidity, and solar radiation. |
| apparent_temperature_min               | The minimum apparent temperature.                                                                            |
| apparent_temperature_mean              | The mean apparent temperature.                                                                               |
| sunrise                                | The time of sunrise.                                                                                         |
| sunset                                 | The time of sunset.                                                                                          |
| precipitation_sum                      | The total precipitation (rain, showers, snow) sum.                                                            |
| rain_sum                               | The sum of only liquid precipitation (rain) of the preceding hour, including local showers and rain from large scale systems.                                             |
| snowfall_sum                           | The sum of snowfall amount of the preceding hour in centimeters. For the water equivalent in millimeters, divide by 7.                                                      |
| precipitation_hours                    | The number of hours of precipitation.                                                                        |
| windspeed_10m_max                      | The maximum wind speed at 10 meters above ground. Wind speed on 10 meters is the standard level.             |
| windgusts_10m_max                      | The maximum gusts at 10 meters above ground of the indicated hour. Wind gusts in CERRA are defined as the maximum wind gusts of the preceding hour. Please consult the ECMWF IFS documentation for more information on how wind gusts are parameterized in weather models. |
| winddirection_10m_dominant             | The dominant wind direction at 10 meters above ground.                                                        |
| shortwave_radiation_sum                | The sum of shortwave solar radiation as the average of the preceding hour. This is equal to the total global horizontal irradiation.                               |
| et0_fao_evapotranspiration             | ET₀ Reference Evapotranspiration of a well-watered grass field. Based on FAO-56 Penman-Monteith equations, ET₀ is calculated from temperature, wind speed, humidity, and solar radiation. Unlimited soil water is assumed. ET₀ is commonly used to estimate the required irrigation for plants. |

### Import Libraries

In [4]:
import requests
import pandas as pd
from pandas import json_normalize
import json
import plotly.express as px
import prophet as pr

### Fetching Weather Data from Open Meteo API

In [6]:
# Define the base URL
url = "https://archive-api.open-meteo.com/v1/archive"

# Define the query parameters as a dictionary
# Time interval: 2010-01-01 to 2023-07-21
# Location: İstanbul
querystring = {
    "latitude": "41.0138",
    "longitude": "28.9497",
    "start_date": "2010-01-01",
    "end_date": "2023-07-21",
    "daily": "weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration",
    "timezone": "auto"
}

# Send a GET request to the URL with the query parameters
response = requests.get(url, params=querystring)

# Print the JSON data returned by the API
print(response.json())

{'latitude': 41.0, 'longitude': 28.90001, 'generationtime_ms': 109.16006565093994, 'utc_offset_seconds': 10800, 'timezone': 'Europe/Istanbul', 'timezone_abbreviation': '+03', 'elevation': 32.0, 'daily_units': {'time': 'iso8601', 'weathercode': 'wmo code', 'temperature_2m_max': '°C', 'temperature_2m_min': '°C', 'temperature_2m_mean': '°C', 'apparent_temperature_max': '°C', 'apparent_temperature_min': '°C', 'apparent_temperature_mean': '°C', 'sunrise': 'iso8601', 'sunset': 'iso8601', 'precipitation_sum': 'mm', 'rain_sum': 'mm', 'snowfall_sum': 'cm', 'precipitation_hours': 'h', 'windspeed_10m_max': 'km/h', 'windgusts_10m_max': 'km/h', 'winddirection_10m_dominant': '°', 'shortwave_radiation_sum': 'MJ/m²', 'et0_fao_evapotranspiration': 'mm'}, 'daily': {'time': ['2010-01-01', '2010-01-02', '2010-01-03', '2010-01-04', '2010-01-05', '2010-01-06', '2010-01-07', '2010-01-08', '2010-01-09', '2010-01-10', '2010-01-11', '2010-01-12', '2010-01-13', '2010-01-14', '2010-01-15', '2010-01-16', '2010-01-

In [15]:
# Extract the JSON content from the response
json_data = response.json()
print(type(json_data['daily']))

<class 'dict'>


In [23]:
# Convert the 'daily' data from the JSON object (json_data) into a DataFrame (df)
df = pd.DataFrame(json_data['daily'])
df

,time,weathercode,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,windspeed_10m_max,windgusts_10m_max,winddirection_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,2010-01-01,1.0,17.8,14.2,15.7,12.6,9.8,10.7,2010-01-01T08:29,2010-01-01T17:46,0.0,0.0,0.00,0.0,42.8,68.0,212.0,7.83,3.20
1,2010-01-02,61.0,15.6,10.3,13.2,10.0,6.7,8.4,2010-01-02T08:29,2010-01-02T17:47,7.7,7.7,0.00,12.0,41.6,64.8,225.0,5.18,1.82
2,2010-01-03,61.0,10.0,4.6,7.3,8.3,0.6,4.2,2010-01-03T08:29,2010-01-03T17:48,18.7,18.7,0.00,23.0,18.9,45.4,17.0,1.91,0.29
3,2010-01-04,71.0,3.9,1.8,2.8,-0.6,-4.0,-2.2,2010-01-04T08:29,2010-01-04T17:48,4.2,3.6,0.49,7.0,27.7,39.6,320.0,6.68,1.12
4,2010-01-05,3.0,7.5,2.4,4.8,3.9,-1.8,0.9,2010-01-05T08:29,2010-01-05T17:49,0.0,0.0,0.00,0.0,12.9,22.3,120.0,8.47,1.31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4945,2023-07-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-17T05:46,2023-07-17T20:34,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4946,2023-07-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-18T05:47,2023-07-18T20:33,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4947,2023-07-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-19T05:48,2023-07-19T20:32,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4948,2023-07-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-07-20T05:48,2023-07-20T20:32,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [30]:
# The NaN values within the dataset are insignificant in terms of their proportion
df.isnull().sum()

df.count()

time                          4950
weathercode                   4945
temperature_2m_max            4945
temperature_2m_min            4945
temperature_2m_mean           4944
apparent_temperature_max      4945
apparent_temperature_min      4945
apparent_temperature_mean     4944
sunrise                       4950
sunset                        4950
precipitation_sum             4944
rain_sum                      4944
snowfall_sum                  4944
precipitation_hours           4950
windspeed_10m_max             4945
windgusts_10m_max             4945
winddirection_10m_dominant    4944
shortwave_radiation_sum       4944
et0_fao_evapotranspiration    4944
dtype: int64

### Data Cleaning

In [33]:
# Convert the 'Weathercode' column to string data type
df['weathercode'] = df['weathercode'].astype(str)

In [ ]:
# Convert the DataFrame to a CSV file
df.to_csv('weather_api.csv', index=False)

### Data Visualization

In [34]:
df.columns

Index(['time', 'weathercode', 'temperature_2m_max', 'temperature_2m_min',
       'temperature_2m_mean', 'apparent_temperature_max',
       'apparent_temperature_min', 'apparent_temperature_mean', 'sunrise',
       'sunset', 'precipitation_sum', 'rain_sum', 'snowfall_sum',
       'precipitation_hours', 'windspeed_10m_max', 'windgusts_10m_max',
       'winddirection_10m_dominant', 'shortwave_radiation_sum',
       'et0_fao_evapotranspiration'],
      dtype='object')

In [39]:
# Creating a line plot using Plotly.express
fig = px.line(df, x='time', y='temperature_2m_mean', title='Mean Temperature at 2m', markers=True)

# Displaying the plot
fig.show()


### Time Series Analysis

In [ ]:
time_df = pd.read_csv("(weather_api.csv")